In [1]:
from fastembed import TextEmbedding

In [3]:
from qdrant_client import QdrantClient, models

In [4]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [5]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [1]:
# documents_raw

In [8]:
from fastembed import TextEmbedding

In [10]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [28]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True, #to get metadata in the results
        with_vectors=True
    )

    return results

In [81]:
q1 = 'I just discovered the course. Can I join now?'

In [95]:
a1 = search(q1)
a1_ans = a1.points[0].payload['text']
a1_vector = a1.points[0].vector # np.array of 512 floats
a1_min_value = np.min(a1_vector)
a1_cos_sim = np.linalg.norm(a1_vector)

In [97]:
a1_min_value

np.float64(-0.10889833)

In [137]:
# q2
q2 = 'Can I still join the course after the start date?'

In [103]:
a2 = search(q2)
a2_ans = a2.points[0].payload['text']
a2_vector = a2.points[0].vector # np.array of 512 floats
a2_min_value = np.min(a2_vector)
a2_cos_sim = np.linalg.norm(a2_vector)

In [128]:
import numpy as np

# Initialize the embedding model
model = TextEmbedding(model_handle)

# Embed query (from q1)
q = next(model.embed([q1]))

# Embed document (q2)
d = next(model.embed([q2]))

# Calculate cosine similarity using dot product
cosine_similarity = q.dot(d)
print(cosine_similarity)

0.9008528895674548


In [138]:
# Q3
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [140]:
# Same embedding model
model = model

# Embed the query (from Q1)
q = q

# Extract text fields from documents
texts = [doc['text'] for doc in documents]

# Embed all document texts
doc_embeddings = list(model.embed(texts))

# Convert to numpy array for easier manipulation
V = np.array(doc_embeddings)

# Calculate cosine similarities using matrix multiplication
similarities = V.dot(q)

# Find the index with highest similarity
best_index = np.argmax(similarities)
print(f"Document index with highest similarity: {best_index}")
print(f"Similarity score: {similarities[best_index]}")

Document index with highest similarity: 1
Similarity score: 0.8182378156620135


In [141]:
# Q4

# Create full_text by concatenating question and text
full_texts = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    full_texts.append(full_text)

# Embed all full_text fields
doc_embeddings = list(model.embed(full_texts))

# Convert to numpy array for matrix operations
V = np.array(doc_embeddings)

# Calculate cosine similarities using matrix multiplication
similarities = V.dot(q)

# Find the index with highest similarity
best_index = np.argmax(similarities)
print(f"Document index with highest similarity: {best_index}")
print(f"Similarity score: {similarities[best_index]}")

# Optional: print all similarities to see the ranking
for i, sim in enumerate(similarities):
    print(f"Document {i}: {sim:.4f}")

Document index with highest similarity: 0
Similarity score: 0.851454319443226
Document 0: 0.8515
Document 1: 0.8437
Document 2: 0.8408
Document 3: 0.7755
Document 4: 0.8086


The answers are different between Q3 and Q4 because different content in each question is embedded, in Q3: Only text field, while in Q4: question + text combined. This changes the semantic meaning and similarity scores. Very broadly speaking, more relevant context will generally lead to better retrieval accuracy.

In [149]:
# Q5
import json

smallest_model = min(TextEmbedding.list_supported_models(), key=lambda m: m["dim"])
smallest_model['dim']

384

In [2]:
# Q6

In [3]:
from qdrant_client import QdrantClient, models

In [4]:
from fastembed import TextEmbedding

In [5]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [6]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [7]:
model_handle = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384

In [ ]:
# Define the collection name
collection_name = "vector-search-q6"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

In [9]:
points = []
id = 0
for doc in documents:
    # Combine question and answer fields as specified
    text = doc['question'] + ' ' + doc['text']
    
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=model_handle), # embed text locally with "BAAI/bge-small-en" from FastEmbed
        payload={
            "text": text,
            "section": doc['section'],
            "course": doc['course']
        } # save all needed metadata fields
    )
    points.append(point)
    id += 1

In [10]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [11]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [13]:
q6 = 'I just discovered the course. Can I join now?'

In [23]:
a6 = search(q6)
# a6_ans = a6.points[0].payload['text']
# a6_vector = a6.points[0].vector # np.array of 512 floats
# a6_min_value = np.min(a6_vector)
# a6_cos_sim = np.linalg.norm(a6_vector)
a6.points[0].score

0.87031734